In [1]:
import numpy as np
import slideio
import cv2
import json
import matplotlib.path as mpltPath
import os
import warnings
import tifffile
from skimage import exposure
from skimage.exposure import match_histograms
import xml.etree.ElementTree as ET
import pandas as pd
from PIL import Image
from math import sqrt
import matplotlib.pyplot as plt
from glob import glob
from copy import copy
from tqdm.notebook import tqdm
import warnings
import staintools

# 경고 메시지를 무시하도록 설정s
warnings.filterwarnings("ignore")

In [ ]:
MMRstatus_data=pd.read_csv('../../data/OriginalData/P53&MMR.csv',encoding='cp949')
def svsToTiff(imageCount):
    global svsWidth,svsHeight,path,scene,fileName
    slide_tile_size=4096
    for widthCount in range(0, int(svsWidth // slide_tile_size)):
        for heightCount in range(0, int(svsHeight // slide_tile_size)):
            points = np.array([path.contains_point(np.array([widthCount* slide_tile_size, heightCount* slide_tile_size])), path.contains_point(np.array([widthCount* slide_tile_size, (heightCount+1)* slide_tile_size])), 
                                 path.contains_point(np.array([(widthCount+1)* slide_tile_size, heightCount* slide_tile_size])),
                                path.contains_point(np.array([(widthCount+1)* slide_tile_size, (heightCount+1)* slide_tile_size]))])
            inside = np.mean(points)>=0.5
            Full=np.mean(points)>=0
            if (inside):
                image = scene.read_block((widthCount * slide_tile_size, heightCount * slide_tile_size, slide_tile_size, slide_tile_size),size=(1024,1024))
                if np.where(image[:,:,1]>=230)[0].shape[0]<(1024*1024)/4:
                    TiffImageSave(image, fileName, imageCount,slide_tile_size)
                    imageCount += 1
            
    return imageCount
def TiffImageSave(image1,fileName,imageCount,slide_tile_size):
    global normalizer2x,normalizer5x,normalizer10x,tmp_img
    img=Image.fromarray(image1) 
    img.save('../../data/ensemble/'+fileName+'_'+str(imageCount)+'.jpg')
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")    
      